In [257]:
import pandas as pd

In [160]:
dates = pd.read_gbq(
    query='select distinct date(deposit_initiated_ts) as date from transactions.fact_deposits',
    project_id='friktion-dev'
    )

In [161]:
dates.describe()

,date
count,99
unique,99
top,2021-12-26
freq,1


In [162]:
users = pd.read_gbq(
    query='select distinct(user_address) as user_address from transactions.fact_deposits')

In [163]:
users.describe()

,user_address
count,12426
unique,12426
top,FrrrZWDn7pCMiGoh35PVZbP8uV2n5nFFJx3uFuyKty2R
freq,1


In [164]:
user_first_deposits_dates = pd.read_gbq(
    query='select user_address, min(date(deposit_initiated_ts)) as first_deposit_dt from transactions.fact_deposits group by user_address')

In [165]:
user_first_deposits_dates.describe()

,user_address,first_deposit_dt
count,12426,12426
unique,12426,99
top,G3fVq24iJjqidFCqbmQqPWvfGvRWzqAyzZFH8iJPjEnP,2022-03-30
freq,1,2160


In [166]:
users.iloc[0][0]

'FrrrZWDn7pCMiGoh35PVZbP8uV2n5nFFJx3uFuyKty2R'

In [168]:
user_first_deposits_dates[user_first_deposits_dates['user_address'].str.contains(users.iloc[0][0])]

,user_address,first_deposit_dt
1863,FrrrZWDn7pCMiGoh35PVZbP8uV2n5nFFJx3uFuyKty2R,2021-12-26


In [73]:
# eventuall you'll iterate through this activity of making a user table per userAddress and then unioning together with all the others

In [171]:
user_first_deposits_dates.shape

(12426, 2)

In [200]:
# to make this iterate through all users increment the first value in the iloc call
# for index in xrange(user_first_deposits_dates.shape[0]):
index = 3
current_user_address = user_first_deposits_dates.iloc[index,:]['user_address']
user_first_deposit_date = user_first_deposits_dates.iloc[index,:]['first_deposit_dt']

In [201]:
user_deposit_table = pd.DataFrame(columns=['user_address', 'as_of_date', 'total_value_locked_USD', 'total_deposited_USD', 'total_withdrawn_USD', \
                                           'tvl_delta_30_days', 'tvl_delta_60_days', 'tvl_delta_90_days', \
                                           'tvl_delta_1_epoch', 'tvl_delta_2_epoch', \
                                           'tvl_delta_3_epoch', 'tvl_delta_4_epoch', 'tvl_delta5_epoch', \
                                           'first_deposit_date', 'first_deposit_epoch', 'first_deposit_amount', 'first_deposit_token', \
                                           'last_deposit_date', 'last_deposit_epoch', 'last_deposit_amount', 'last_deposit_token', \
                                           'days_since_last_deposit', 'epochs_since_last_deposit', \
                                           'first_withdrawal_date', 'first_withdrawal_epoch', 'first_withdrawal_amount', 'first_withdrawal_token', \
                                           'last_withdrawal_date', 'last_withdrawal_epoch', 'last_withdrawal_amount', 'last_withdrawal_token', \
                                           'has_churned', 'churn_date', 'churn_epoch'])

### asOfDate

In [202]:
user_first_deposit_date

datetime.date(2021, 12, 30)

In [203]:
dates[dates['date'].apply(lambda x: x>=user_first_deposit_date)].sort_values('date').iloc[:,0]

5     2021-12-30
4     2021-12-31
9     2022-01-01
13    2022-01-02
10    2022-01-03
         ...    
98    2022-03-28
94    2022-03-29
92    2022-03-30
93    2022-03-31
95    2022-04-01
Name: date, Length: 93, dtype: dbdate

In [204]:
user_deposit_table['as_of_date'] = dates[dates['date'].apply(lambda x: x>=user_first_deposit_date)].sort_values('date').iloc[:,0]

### userAddress

In [205]:
user_deposit_table['user_address'] = current_user_address

### totalValueDepositedUSD
TODO: Add logic to covert to USD

#### inner loop logic

In [213]:
user_deposit_table.shape[0]

93

In [214]:
user_deposit_table.index = range(user_deposit_table.shape[0])

In [215]:
query_date = user_deposit_table['as_of_date'][0].isoformat()
pd.read_gbq(query=f"select sum(deposit_initiated_amt) as total_deposited from transactions.fact_deposits where user_address=\'{current_user_address}\' and date(deposit_initiated_ts) <= date(\'{query_date}\')")

,total_deposited
0,0.060406


In [216]:
# need to add market marking code
user_deposit_table['total_deposited_USD'] = [pd.read_gbq(query=f"select sum(deposit_initiated_amt) as total_deposited from transactions.fact_deposits where user_address=\'{current_user_address}\' and date(deposit_initiated_ts) <= date(\'{query_date}\')").iloc[0,0] for query_date in user_deposit_table['as_of_date'].tolist()]

### totalValueWithdrawnUSD
TODO: Add logic to convert to USD

In [222]:
query_date = user_deposit_table['as_of_date'][0].isoformat()
pd.read_gbq(query=f"select sum(withdrawal_initiated_amt) as total_withdrawn from transactions.fact_withdrawals where user_address=\'{current_user_address}\' and date(withdrawal_initiated_ts) <= date(\'{query_date}\')").fillna(0).iloc[0,0]

0.0

In [228]:
# need to add market marking code
# will need to change to the usd amount from the trans tables in prod
user_deposit_table['total_withdrawn_USD'] = [pd.read_gbq(query=f"select sum(withdrawal_initiated_amt) as total_withdrawn from transactions.fact_withdrawals where user_address=\'{current_user_address}\' and date(withdrawal_initiated_ts) <= date(\'{query_date}\')").fillna(0).iloc[0,0] for query_date in user_deposit_table['as_of_date'].tolist()]

### totalValueLockedUSD

In [229]:
user_deposit_table['total_value_locked_USD'] = user_deposit_table['total_deposited_USD'] - user_deposit_table['total_withdrawn_USD']

### firstDepositDate

In [231]:
user_deposit_table['first_deposit_date'] =  user_first_deposit_date

### firstDepositEpoch

In [237]:
user_deposit_table['first_deposit_epoch'] = pd.read_gbq(query=f'select deposit_initiated_epoch from transactions.fact_deposits where date(deposit_initiated_ts) = \'{user_first_deposit_date}\'')

### firstDepositAmount

In [241]:
user_deposit_table['first_deposit_amount'] = pd.read_gbq(query=f'select sum(deposit_initiated_amt) from transactions.fact_deposits where user_address = \'{current_user_address}\' and date(deposit_initiated_ts) = \'{user_first_deposit_date.isoformat()}\'').iloc[0,0]

### firstDepositToken

In [242]:
user_deposit_table['first_deposit_token'] = pd.read_gbq(query=f'select deposited_asset from transactions.fact_deposits where user_address = \'{current_user_address}\' and date(deposit_initiated_ts) = \'{user_first_deposit_date.isoformat()}\'').iloc[0,0]

### tvlDelta30Days

In [249]:
current_as_of_date = user_deposit_table['as_of_date'].iloc[3]

# we can also perform this logic with GBQ SQL
#      DATE_ADD(DATE "2022-01-19", INTERVAL 30 DAY)
window_end_date = current_as_of_date - datetime.timedelta(30)

q = f'\
    with window_deposits as \
         (select user_address, IFNULL(sum(deposit_initiated_amt), 0.0) as deposits \
         from transactions.fact_deposits \
         where user_address = \'{current_user_address}\' \
         and date(deposit_initiated_ts) between \'{window_end_date.isoformat()}\' and \'{current_as_of_date.isoformat()}\'\
         group by user_address) \
    select * from window_deposits'

r = pd.read_gbq(query=q)
r

,user_address,deposits
0,D8ayPyamAg1yWyqCeG3u2L2fY2r6XmxfS8fdjf4wA1qx,18.060406


In [251]:
tvl_list = []
for table_date in user_deposit_table['as_of_date']:
    current_as_of_date = table_date
    # we can also perform this logic with GBQ SQL
    #      DATE_ADD(DATE "2022-01-19", INTERVAL -30 DAY)
    window_end_date = current_as_of_date - datetime.timedelta(30)
    window_delta = pd.read_gbq(query=f'\
        with window_deposits as \
         (select user_address, IFNULL(sum(deposit_initiated_amt), 0.0) as deposits \
         from transactions.fact_deposits \
         where user_address = \'{current_user_address}\' \
         and date(deposit_initiated_ts) between \'{window_end_date}\' and \'{current_as_of_date.isoformat()}\'\
             group by user_address) \
        , window_withdrawals as \
            (select user_address, sum(withdrawal_initiated_amt) as withdrawals \
            from transactions.fact_withdrawals \
            where user_address = \'{current_user_address}\' \
            and date(withdrawal_initiated_ts) between \'{window_end_date}\' and \'{current_as_of_date.isoformat()}\'\
            group by user_address) \
        select \
             IFNULL(deposits, 0.0) - IFNULL(withdrawals, 0.0) \
        from window_deposits wd\
        left join window_withdrawals ww on wd.user_address = ww.user_address \
        ')
    if window_delta.empty:
        # print(window_end_date, current_as_of_date)
        # print(table_date, window_delta.empty)
        tvl_list.append(0.0)
    else:
        # print(window_end_date, current_as_of_date)
        # print(window_delta)
        # print(table_date, window_delta.iloc[0,0])
        tvl_list.append(window_delta.iloc[0,0])
    
user_deposit_table['tvl_delta_30_days'] = tvl_list

### tvlDelta60Days

In [254]:
tvl_list = []
for table_date in user_deposit_table['as_of_date']:
    current_as_of_date = table_date
    # we can also perform this logic with GBQ SQL
    #      DATE_ADD(DATE "2022-01-19", INTERVAL -30 DAY)
    window_end_date = current_as_of_date - datetime.timedelta(60)
    window_delta = pd.read_gbq(query=f'\
        with window_deposits as \
         (select user_address, IFNULL(sum(deposit_initiated_amt), 0.0) as deposits \
         from transactions.fact_deposits \
         where user_address = \'{current_user_address}\' \
         and date(deposit_initiated_ts) between \'{window_end_date}\' and \'{current_as_of_date.isoformat()}\'\
             group by user_address) \
        , window_withdrawals as \
            (select user_address, sum(withdrawal_initiated_amt) as withdrawals \
            from transactions.fact_withdrawals \
            where user_address = \'{current_user_address}\' \
            and date(withdrawal_initiated_ts) between \'{window_end_date}\' and \'{current_as_of_date.isoformat()}\'\
            group by user_address) \
        select \
             IFNULL(deposits, 0.0) - IFNULL(withdrawals, 0.0) \
        from window_deposits wd\
        left join window_withdrawals ww on wd.user_address = ww.user_address \
        ')
    if window_delta.empty:
        # print(window_end_date, current_as_of_date)
        # print(table_date, window_delta.empty)
        tvl_list.append(0.0)
    else:
        # print(window_end_date, current_as_of_date)
        # print(window_delta)
        # print(table_date, window_delta.iloc[0,0])
        tvl_list.append(window_delta.iloc[0,0])

user_deposit_table['tvl_delta_60_days'] = tvl_list

### tvlDelta90Days

In [268]:
tvl_list = []
for table_date in user_deposit_table['as_of_date']:
    current_as_of_date = table_date
    # we can also perform this logic with GBQ SQL
    #      DATE_ADD(DATE "2022-01-19", INTERVAL -30 DAY)
    window_end_date = current_as_of_date - datetime.timedelta(90)
    window_delta = pd.read_gbq(query=f'\
        with window_deposits as \
         (select user_address, IFNULL(sum(deposit_initiated_amt), 0.0) as deposits \
         from transactions.fact_deposits \
         where user_address = \'{current_user_address}\' \
         and date(deposit_initiated_ts) between \'{window_end_date}\' and \'{current_as_of_date.isoformat()}\'\
             group by user_address) \
        , window_withdrawals as \
            (select user_address, sum(withdrawal_initiated_amt) as withdrawals \
            from transactions.fact_withdrawals \
            where user_address = \'{current_user_address}\' \
            and date(withdrawal_initiated_ts) between \'{window_end_date}\' and \'{current_as_of_date.isoformat()}\'\
            group by user_address) \
        select \
             IFNULL(deposits, 0.0) - IFNULL(withdrawals, 0.0) \
        from window_deposits wd\
        left join window_withdrawals ww on wd.user_address = ww.user_address \
        ')
    if window_delta.empty:
        # print(window_end_date, current_as_of_date)
        # print(table_date, window_delta.empty)
        tvl_list.append(0.0)
    else:
        # print(window_end_date, current_as_of_date)
        # print(window_delta)
        # print(table_date, window_delta.iloc[0,0])
        tvl_list.append(window_delta.iloc[0,0])
    
user_deposit_table['tvl_delta_90_days'] = tvl_list

### tvlDelta1Epoch

### tvlDelta2Epoch

### tvlDelta3Epoch

### tvlDelta4Epoch

### tvlDelta5Epoch

### lastDepositDate

In [284]:
pd.read_gbq(query=f'select max(date(deposit_initiated_ts)) from transactions.fact_deposits where user_address = \'{current_user_address}\' and date(deposit_initiated_ts) <= \'{current_as_of_date}\'')

,f0_
0,2021-12-31


In [285]:
last_deposit_list = []

for table_date in user_deposit_table['as_of_date']:
    current_as_of_date = table_date
    r = pd.read_gbq(query=f'select max(date(deposit_initiated_ts)) from transactions.fact_deposits where user_address = \'{current_user_address}\' and date(deposit_initiated_ts) <= \'{current_as_of_date}\'')
    last_deposit_list.append(r.iloc[0,0])
user_deposit_table['last_deposit_date'] = last_deposit_list

### lastDepositEpoch

In [286]:
last_deposit_epoch_list = []

for table_date in user_deposit_table['as_of_date']:
    current_as_of_date = table_date
    r = pd.read_gbq(query=f'select max((deposit_initiated_epoch)) from transactions.fact_deposits where date(deposit_initiated_ts) <= \'{current_as_of_date}\'')
    last_deposit_epoch_list.append(r.iloc[0,0])
user_deposit_table['last_deposit_epoch'] = last_deposit_epoch_list

### lastDepositAmount & lastDepositToken

In [303]:
last_deposit_amount_list = []
last_deposit_token_list = []

for table_date in user_deposit_table['last_deposit_date']:
    r = pd.read_gbq(query=f'select deposited_asset, deposit_initiated_amt from transactions.fact_deposits where user_address = \'{current_user_address}\' and date(deposit_initiated_ts) = \'{table_date}\'')
    
    last_deposit_token_list.append(r.iloc[0,0])
    last_deposit_amount_list.append(r.iloc[0,1])
user_deposit_table['last_deposit_token'] = last_deposit_token_list
user_deposit_table['last_deposit_amount'] = last_deposit_amount_list

### daysSinceLastDeposit

In [288]:
days_since_last_deposit_list = []
for as_of_date, last_deposit_date in user_deposit_table[['as_of_date', 'last_deposit_date']].to_records(index=False).tolist():
    days_since_last_deposit_list.append((as_of_date - last_deposit_date).days)
user_deposit_table['days_since_last_deposit'] = days_since_last_deposit_list

### epochsSinceLastDeposit

### firstWithdrawalDate

In [270]:
r = pd.read_gbq(query=f'select min(date(withdrawal_initiated_ts)) from transactions.fact_withdrawals where user_address = \'{current_user_address}\'')
if r.empty:
    pass
else:
    user_deposit_table['first_withdrawal_date'] = r.iloc[0,0]

### firstWithdrawalEpoch

In [326]:
first_withdrawal_date = user_deposit_table['first_withdrawal_date'].iloc[0]
r = pd.read_gbq(query=f'select (withdrawal_initiated_epoch) from transactions.fact_withdrawals where date(withdrawal_initiated_ts) <= \'{first_withdrawal_date}\'')
user_deposit_table['first_withdrawal_epoch'] = r.iloc[0,0]

### firstWithdrawalAmount & firstWithdrawalToken

In [272]:
first_withdrawal_date = user_deposit_table['first_withdrawal_date'].iloc[0]
r = pd.read_gbq(query=f'select deposited_asset, withdrawal_initiated_amt from transactions.fact_withdrawals where user_address = \'{current_user_address}\' and date(withdrawal_initiated_ts) = \'{first_withdrawal_date}\'')
r
user_deposit_table['first_withdrawal_token'] = r.iloc[0,0]
user_deposit_table['first_withdrawal_amount'] = r.iloc[0,1]

### lastWithdrawalDate

In [273]:
last_withdrawal_list = []

for table_date in user_deposit_table['as_of_date']:
    current_as_of_date = table_date
    r = pd.read_gbq(query=f'select max(date(withdrawal_initiated_ts)) from transactions.fact_withdrawals where user_address = \'{current_user_address}\' and date(withdrawal_initiated_ts) <= \'{current_as_of_date}\'')
    last_withdrawal_list.append(r.iloc[0,0])
user_deposit_table['last_withdrawal_date'] = last_withdrawal_list

### lastWithdrawalEpoch

In [338]:
if pd.isnull(user_deposit_table['last_withdrawal_date'].iloc[15]):
    print('its true')
else:
    print('its false')

its false


In [339]:
last_withdrawal_epoch_list = []

for table_date in user_deposit_table['last_withdrawal_date']:
    if pd.isnull(table_date):
        last_withdrawal_epoch_list.append(None)
    else:
        r = pd.read_gbq(query=f'select max((withdrawal_initiated_epoch)) from transactions.fact_withdrawals where user_address = \'{current_user_address}\' and date(withdrawal_initiated_ts) = \'{table_date}\'')
        last_withdrawal_epoch_list.append(r.iloc[0,0])
user_deposit_table['last_withdrawal_epoch'] = last_withdrawal_epoch_list

### lastWithdrawalAmount & lastWithdrawalToken

In [274]:
last_withdrawal_amount_list = []
last_withdrawal_token_list = []

for table_date in user_deposit_table['last_withdrawal_date']:
    current_as_of_date = table_date
    # print(current_as_of_date)
    if pd.isnull(current_as_of_date):
        last_withdrawal_token_list.append(None)
        last_withdrawal_amount_list.append(None)
    else:
        r = pd.read_gbq(query=f'select deposited_asset, withdrawal_initiated_amt from transactions.fact_withdrawals where user_address = \'{current_user_address}\' and date(withdrawal_initiated_ts) = \'{current_as_of_date}\'')

        last_withdrawal_token_list.append(r.iloc[0,0])
        last_withdrawal_amount_list.append(r.iloc[0,1])
user_deposit_table['last_withdrawal_token'] = last_withdrawal_token_list
user_deposit_table['last_withdrawal_amount'] = last_withdrawal_amount_list

NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
NaT
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08
2022-01-08


### hasChurned

In [307]:
user_deposit_table['has_churned'] = (user_deposit_table['total_deposited_USD'] - user_deposit_table['total_withdrawn_USD']).apply(lambda x: x <= 0)

### churnDate

In [322]:
churn_state = user_deposit_table['has_churned'].tolist()
if any(churn_state):
    user_deposit_table['churn_date'] = None
else:
    churn_date_list = []

    for index in range(len(churn_state)-1):
        if index == 0:
            churn_date_list.append(None)
        if (churn_state[index-1] == False ) & (churn_state[index] == True):
            churn_date_list.append(user_deposit_table['as_of_date'].iloc[index])
        else:
            churn_date_list.append(churn_date_list[index-1])

    user_deposit_table['churn_date'] = churn_date_list

### churnEpoch

# Table Peeks

In [283]:
pd.read_gbq(query=f'select deposit_initiated_ts, deposit_initiated_amt from transactions.fact_deposits where user_address = \'{current_user_address}\'')

,deposit_initiated_ts,deposit_initiated_amt
0,2021-12-30 04:25:57+00:00,0.060406
1,2021-12-31 16:19:15+00:00,18.000000


In [289]:
user_deposit_table.iloc[:5,:][['as_of_date', 'last_deposit_date']]

,as_of_date,last_deposit_date
0,2021-12-30,2021-12-30
1,2021-12-31,2021-12-31
2,2022-01-01,2021-12-31
3,2022-01-02,2021-12-31
4,2022-01-03,2021-12-31


In [340]:
user_deposit_table.iloc[17,:]

user_address                 D8ayPyamAg1yWyqCeG3u2L2fY2r6XmxfS8fdjf4wA1qx
as_of_date                                                     2022-01-16
total_value_locked_USD                                          18.003346
total_deposited_USD                                             18.060406
total_withdrawn_USD                                               0.05706
tvl_delta_30_days                                               18.003346
tvl_delta_60_days                                               18.003346
tvl_delta_90_days                                               18.003346
tvl_delta_1_epoch                                                     NaN
tvl_delta_2_epoch                                                     NaN
tvl_delta_3_epoch                                                     NaN
tvl_delta_4_epoch                                                     NaN
tvl_delta5_epoch                                                      NaN
first_deposit_date                    